# Decision Tree Markov Chain

Numerical example from:<br>
[DTMC: An Actionable e-Customer Lifetime Value ModelBased on Markov Chains and Decision Trees](http://liacs.leidenuniv.nl/~puttenpwhvander/library/200706icec75-paauwe.pdf)

In [1]:
import numpy as np
import pandas as pd

In [2]:
customers_raw = pd.read_csv('dtmc_data.csv')
customers_raw.head()

,customer,period,recency,frequency,monetary
0,1,1,0.0,1.0,10.0
1,1,2,1.0,0.0,0.0
2,1,3,2.0,0.0,0.0
3,1,4,3.0,0.0,0.0
4,1,5,4.0,0.0,0.0


### Segmentation using decision tree

In [3]:
customers = customers_raw[
    (customers_raw['frequency'] > 0) | ((customers_raw['frequency']==0) & (customers_raw['recency'] == 6))
].copy()

In [4]:
def tree(df):
    "Same tree as used in the paper above"
    if df['frequency'] == 0 and df['recency'] == 6:
        return 5
    else:
        if df['frequency'] <= 2:
            if df['recency'] == 0:
                return 1
            else:
                return 2
        else:
            if df['recency'] == 0:
                return 3
            else:
                return 4

In [5]:
customers.loc[:,'segment'] = customers.apply(tree, axis=1)
customers.head()

,customer,period,recency,frequency,monetary,segment
0,1,1,0.0,1.0,10.0,1
6,1,7,6.0,0.0,0.0,5
16,2,1,0.0,2.0,40.0,1
21,2,6,5.0,2.0,100.0,2
25,2,10,4.0,1.0,50.0,2


In [6]:
# compute contribution - average monetary within segment
contribution = customers.groupby('segment').mean()['monetary']
# add begin state
contribution = contribution.append(pd.Series(index=[0], data=[0]))
contribution = contribution.sort_index()
contribution

0      0.0
1     25.0
2     75.0
3    100.0
4    200.0
5      0.0
dtype: float64

### Compute transition probabilities

In [7]:
temp1 = customers.copy()
# add 0th period for each customer
for customer in customers['customer'].unique():
    temp1 = temp1.append(
        pd.Series(index=temp1.columns, data=[customer, 0,0,0,0,0]),
        ignore_index=True
    )

temp1['pseudo_period'] = temp1.groupby('customer').rank(method='min')['period']
temp2 = temp1.copy()
temp2['pseudo_period'] = temp2['pseudo_period'] - 1

temp1 = temp1.set_index(['customer', 'pseudo_period'])
temp2 = temp2.set_index(['customer', 'pseudo_period'])
temp = temp1.join(temp2, rsuffix='_r', how='inner')[['segment', 'segment_r']]

transitions = pd.DataFrame(
    index=range(6),
    columns=range(6),
    data=np.zeros((6, 6))
)

for row in temp.values:
    transitions.loc[row[0]][row[1]] += 1

In [8]:
# observed transitions
transitions

,0,1,2,3,4,5
0,0.0,2.0,0.0,2.0,0.0,0.0
1,0.0,0.0,1.0,0.0,0.0,1.0
2,0.0,0.0,1.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,0.0,2.0
4,0.0,0.0,1.0,0.0,1.0,0.0
5,0.0,0.0,0.0,0.0,1.0,0.0


In [9]:
# transiton probabilities assuming 5 is an absorbing state
transision_proba = transitions.div(transitions.sum(axis=1), axis=0)
transision_proba.loc[5] = [0,0,0,0,0,1]

transision_proba

,0,1,2,3,4,5
0,0.0,0.5,0.0,0.5,0.0,0.0
1,0.0,0.0,0.5,0.0,0.0,0.5
2,0.0,0.0,0.5,0.0,0.0,0.5
3,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.5,0.0,0.5,0.0
5,0.0,0.0,0.0,0.0,0.0,1.0


### Compute CLV per segment

In [10]:
T = 25
d = 0.01

In [11]:
# accounting for different period
N = 2.7 #average between-purchase-period

T_new = round(T/N)
d_new = (1 + d)**(T/N)-1

In [12]:
def CLV(P, R, T, d):
    """Computes CLV using MC for T steps
    
    :param P: transition matrix
    :param R: reward vector
    :param T: number of steps
    :param d: discount factor
    
    :returns: CLV vector
    """
    CLV = pd.Series(index=R.index,
                    data=np.zeros(len(R))
                   )
    for t in range(1, T+1):
        clv = np.linalg.matrix_power(P.values/(1+d), t)*R.values
        CLV += clv.sum(axis=1)
    return CLV

In [13]:
CLV(transision_proba, contribution, T_new, d_new)

0     85.611656
1     62.812053
2     62.812053
3      0.000000
4    282.478205
5      0.000000
dtype: float64